In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
from glob import glob

In [2]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [4]:
def draw_landmarks(image, results):
#     mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACE_CONNECTIONS) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [5]:
def draw_styled_landmarks(image, results):
    # Draw face connections
#     mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACE_CONNECTIONS, 
#                              mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
#                              mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
#                              ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [6]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    #face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, lh, rh])

In [7]:
def create_dir(path):
    try:
        if not os.path.exists(path):
            os.makedirs(path)
    except OSError:
        print(f"ERROR: creating directory with name {path}")

In [10]:
def save_frame(video_path, save_dir, gap):
    name = video_path.split("\\")[-1].split(".")[0]
    save_path = os.path.join(save_dir, name)
    create_dir(save_path)
    print(name)
    cap = cv2.VideoCapture(video_path)
    idx = 0
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        while True:
            ret, frame = cap.read()
            
            if ret == True:
                image, results = mediapipe_detection(frame, holistic)
                draw_styled_landmarks(image, results)

            if ret == False:
                cap.release()
                break
            
            if idx == 0:
                #cv2.imwrite(f"{save_path}/{idx}.png", image)
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(save_path, str(idx))
                np.save(npy_path, keypoints)
            else:
                if idx % gap == 0:
                    #cv2.imwrite(f"{save_path}/{idx}.png", image)
                    keypoints = extract_keypoints(results)
                    npy_path = os.path.join(save_path, str(int(idx)))
                    np.save(npy_path, keypoints)
            #print(keypoints)
            idx += 1

In [13]:
video_paths = glob("M:\\Work\\MIT\\DatasetV3\\dataset-v2-npy-split\\stand\\*")
#video_paths = glob("videos\\*")
save_dir = "save\stand"
for path in video_paths:
    save_frame(path, save_dir, gap=1)
    print(path)

0
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\0.mp4
1
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\1.mp4
10
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\10.mp4
100
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\100.mp4
1000
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\1000.mp4
1001
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\1001.mp4
1002
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\1002.mp4
1003
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\1003.mp4
1004
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\1004.mp4
1005
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\1005.mp4
1006
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\1006.mp4
1007
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\1007.mp4
1008
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\1008.mp4
1009
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\1009.mp4
101
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\101.mp4
1010
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\1010.mp4
1011
M:\Work

M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\1115.mp4
1116
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\1116.mp4
1117
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\1117.mp4
1118
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\1118.mp4
1119
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\1119.mp4
112
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\112.mp4
1120
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\1120.mp4
1121
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\1121.mp4
1122
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\1122.mp4
1123
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\1123.mp4
1124
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\1124.mp4
1125
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\1125.mp4
1126
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\1126.mp4
1127
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\1127.mp4
1128
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\1128.mp4
1129
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\1129.mp4

M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\1233.mp4
1234
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\1234.mp4
1235
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\1235.mp4
1236
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\1236.mp4
1237
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\1237.mp4
1238
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\1238.mp4
1239
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\1239.mp4
124
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\124.mp4
1240
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\1240.mp4
1241
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\1241.mp4
1242
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\1242.mp4
1243
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\1243.mp4
1244
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\1244.mp4
1245
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\1245.mp4
1246
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\1246.mp4
1247
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\1247.mp4

M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\1351.mp4
1352
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\1352.mp4
1353
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\1353.mp4
1354
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\1354.mp4
1355
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\1355.mp4
1356
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\1356.mp4
1357
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\1357.mp4
1358
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\1358.mp4
1359
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\1359.mp4
136
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\136.mp4
1360
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\1360.mp4
1361
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\1361.mp4
1362
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\1362.mp4
1363
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\1363.mp4
1364
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\1364.mp4
1365
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\1365.mp4

M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\147.mp4
1470
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\1470.mp4
1471
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\1471.mp4
1472
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\1472.mp4
1473
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\1473.mp4
1474
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\1474.mp4
1475
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\1475.mp4
1476
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\1476.mp4
1477
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\1477.mp4
1478
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\1478.mp4
1479
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\1479.mp4
148
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\148.mp4
1480
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\1480.mp4
1481
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\1481.mp4
1482
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\1482.mp4
1483
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\1483.mp4


...

M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\466.mp4
467
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\467.mp4
468
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\468.mp4
469
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\469.mp4
47
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\47.mp4
470
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\470.mp4
471
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\471.mp4
472
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\472.mp4
473
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\473.mp4
474
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\474.mp4
475
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\475.mp4
476
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\476.mp4
477
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\477.mp4
478
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\478.mp4
479
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\479.mp4
48
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\48.mp4
480
M:\Work\MIT\DatasetV3\datase

M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\588.mp4
589
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\589.mp4
59
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\59.mp4
590
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\590.mp4
591
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\591.mp4
592
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\592.mp4
593
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\593.mp4
594
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\594.mp4
595
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\595.mp4
596
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\596.mp4
597
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\597.mp4
598
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\598.mp4
599
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\599.mp4
6
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\6.mp4
60
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\60.mp4
600
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\600.mp4
601
M:\Work\MIT\DatasetV3\dataset-v2

M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\709.mp4
71
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\71.mp4
710
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\710.mp4
711
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\711.mp4
712
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\712.mp4
713
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\713.mp4
714
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\714.mp4
715
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\715.mp4
716
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\716.mp4
717
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\717.mp4
718
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\718.mp4
719
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\719.mp4
72
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\72.mp4
720
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\720.mp4
721
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\721.mp4
722
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\722.mp4
723
M:\Work\MIT\DatasetV3\datase

M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\830.mp4
831
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\831.mp4
832
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\832.mp4
833
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\833.mp4
834
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\834.mp4
835
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\835.mp4
836
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\836.mp4
837
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\837.mp4
838
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\838.mp4
839
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\839.mp4
84
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\84.mp4
840
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\840.mp4
841
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\841.mp4
842
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\842.mp4
843
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\843.mp4
844
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\844.mp4
845
M:\Work\MIT\DatasetV3\data

M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\952.mp4
953
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\953.mp4
954
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\954.mp4
955
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\955.mp4
956
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\956.mp4
957
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\957.mp4
958
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\958.mp4
959
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\959.mp4
96
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\96.mp4
960
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\960.mp4
961
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\961.mp4
962
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\962.mp4
963
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\963.mp4
964
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\964.mp4
965
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\965.mp4
966
M:\Work\MIT\DatasetV3\dataset-v2-npy-split\stand\966.mp4
967
M:\Work\MIT\DatasetV3\data